# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Usage of deprecated web service protocols

This notebook contains Kusto queries that can help determining if one ore more environments are still using web services build on the depreated Odata v3 protocol. 

NB! The signal used in this notebook is only available in versions 16.1 (or newer) of Business Central, so check the version of your environment if you don't see any data.

Web service request signal is documented here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace 

## 1. Connect to Application Insights
First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [13]:
# load the KQLmagic module
%reload_ext Kqlmagic

# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'

## 2. Define filters
This workbook is designed for troubleshooting environments in an AAD subscription. Please provide values for aadTenantId below: 

In [14]:
aadTenantId = "<Add AAD tenant id here>"

# Analyze usage of web services

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

## Request stats (last 30 days) - all categories

In [15]:
%%kql
let _aadTenantId = aadTenantId;
traces
| where 1==1 
    and timestamp > ago(30d)
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.eventId == 'RT0008'
| extend category = tostring( customDimensions.category )
| summarize request_count=count() by category, bin(timestamp, 1d)
| render timechart title= 'Number of web service requests by category (ODatav4, ODatav3, SOAP, API)'

In [16]:
%%kql
let _aadTenantId = aadTenantId;
traces
| where 1==1 
    and timestamp > ago(30d)
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.eventId == 'RT0008'
    and customDimensions.category == 'ODataV3'
| extend category = tostring( customDimensions.category )
| summarize request_count=count() by category, bin(timestamp, 1d)
| render timechart title= 'Number of web service requests by category (deprecated categories)'

## Web service endpoints using deprecated protocols

The following endpoints are exposed ad web service endpoints using a deprecated protocol (query shows the first 100)

In [17]:
%%kql
let _aadTenantId = aadTenantId;
traces
| where 1==1 
    and timestamp > ago(30d)
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.eventId == 'RT0008'
    and customDimensions.category == 'ODataV3'
| project aadId = tostring( customDimensions.aadTenantId )
        , environmentName = tostring( customDimensions.environmentName )
        , category = tostring( customDimensions.category )
        , endpoint = tostring( customDimensions.endpoint )
| distinct aadId
        , environmentName 
        , category 
        , endpoint
| limit 100